In [1]:
%%configure -f
{ "conf":{    
    "spark.jars.packages": "io.delta:delta-core_2.12:0.7.0,com.databricks:spark-xml_2.10:0.2.0",
    "spark.sql.autoBroadcastJoinThreshold": "-1",
    "spark.sql.extensions": "io.delta.sql.DeltaSparkSessionExtension",
    "spark.sql.catalog.spark_catalog": "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    "spark.databricks.delta.retentionDurationCheck.enabled": "False",
    "spark.databricks.delta.vacuum.parallelDelete.enabled": "True",
    "spark.databricks.delta.schema.autoMerge.enabled": "True",
    "spark.sql.adaptive.enabled":"True",
    "spark.sql.legacy.parquet.datetimeRebaseModeInRead":"LEGACY",
    "spark.sql.legacy.parquet.datetimeRebaseModeInWrite":"CORRECTED",
    "spark.sql.parquet.int96RebaseModeInWrite":"CORRECTED",
    "spark.sql.parquet.int96RebaseModeInRead": "CORRECTED",
    "spark.executor.memory": "70g",
    "spark.driver.memory": "50g",
    "spark.memory.offHeap.enabled": "true",
    "spark.memory.offHeap.size": "16g",
    "spark.sql.shuffle.partitions":"256"
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1748427205490_0001,pyspark,idle,Link,Link,None,


In [16]:
# THIS CURRENT 'EMR_WORKSPACE_ID' and 'WORKSPACE_LOCATION' are based on account used in AWS
# So, these values must be updated based on account and role used for testing it

# EMR_WORKSPACE_ID is the ID for the workspace you created. So, modify this value by getting this value in the following section:
# EMR Studio: Workspaces (Notebooks) "<YOUR WORKSPACE NAME CREATED>":
# In section "Workspace details Info"
# Verify the id in "Workspace ID"
EMR_WORKSPACE_ID = "e-8NOXK1K4GBFTWHTB3NSR1YUZL" # Change this value

#The workspace location you can get from "Workspace storage location" section
#"Workspace storage location" s3://aws-emr-resources-271380526857-us-west-2/notebooks/
WORKSPACE_LOCATION = "s3://aws-emr-resources-271380526857-us-west-2/notebooks" # # Change this value

sc.addPyFile(
    f"{WORKSPACE_LOCATION}/{EMR_WORKSPACE_ID}/breweries.zip"
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from breweries.services.bronze.BronzeService import BronzeService
from breweries.services.silver.SilverService import SilverService
from breweries.services.gold.GoldService import GoldService
from breweries.common.medallion import Bronze, Silver, Gold

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# How to run the Bronze Service
BronzeService().run()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Gets Spark Session
Additional Parameters: 
{'spark.sql.autoBroadcastJoinThreshold': '-1', 'spark.executor.memory': '70g', 'spark.driver.memory': '50g', 'spark.memory.offHeap.enabled': 'true', 'spark.memory.offHeap.size': '16g', 'spark.driver.extraJavaOptions': '-Xms20g', 'spark.databricks.delta.schema.autoMerge.enabled': 'true'}
pyspark is in /mnt3/yarn/usercache/livy/appcache/application_1748427205490_0003/container_1748427205490_0003_01_000001/pyspark.zip/pyspark/__init__.py

In [5]:
# How to run the Silver Service
SilverService().run()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# How to run the Gold Service
GoldService().run()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# Reading Bronze data from Medallion architecture
Bronze().read_parquet(spark).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------------------------+------------+--------------------+--------------------+----------+------------+-------------------+-----------+----------------+---------------+----------------+--------------------+-------------------+--------------------+-----------+
|                  id|                         name|brewery_type|           address_1|           address_2| address_3|        city|     state_province|postal_code|       longitude|       latitude|           phone|         website_url|              state|              street|    country|
+--------------------+-----------------------------+------------+--------------------+--------------------+----------+------------+-------------------+-----------+----------------+---------------+----------------+--------------------+-------------------+--------------------+-----------+
|cfa85b4b-30b6-477...| 에잇피플브루어리(Eight Pe...|     brewpub|3, Jaejaegi-ro 19...|           Hwado-eup|      null|Namyangju-si|         Gyeo

In [11]:
# Reading Silver data from Medallion architecture
Silver().read_parquet(spark).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------------------------+------------+--------------------+--------------------+----------+------------+----------------+-----------+----------------+---------------+----------------+--------------------+----------------+--------------------+-----------+
|                  id|                         name|brewery_type|           address_1|           address_2| address_3|        city|  state_province|postal_code|       longitude|       latitude|           phone|         website_url|           state|              street|    country|
+--------------------+-----------------------------+------------+--------------------+--------------------+----------+------------+----------------+-----------+----------------+---------------+----------------+--------------------+----------------+--------------------+-----------+
|cfa85b4b-30b6-477...| 에잇피플브루어리(Eight Pe...|     brewpub|3, Jaejaegi-ro 19...|           Hwado-eup|       N/A|Namyangju-si|      Gyeonggido|      12200|  

In [13]:
# Reading Gold data from Medallion architecture
(Gold() / "agg_by_brewery_type").read_parquet(spark).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+------------+
|count|brewery_type|
+-----+------------+
|    6|        nano|
|    3|    contract|
|    9|      closed|
|    3|  proprietor|
|   27|     taproom|
|  435|       micro|
|   63|       large|
|   21|    regional|
|   90|         bar|
|  273|     brewpub|
+-----+------------+

In [15]:
# Reading Gold data from Medallion architecture
(Gold() / "agg_by_country").read_parquet(spark).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-------------+
|count|      country|
+-----+-------------+
|  102|       Poland|
|    6|  Isle of Man|
|    9|       France|
|    3|       France|
|    3|       France|
|  150|      Ireland|
|   50|      Ireland|
|   50|      Ireland|
|   99|    Singapore|
|   33|    Singapore|
|   33|    Singapore|
|   30|     Scotland|
|   10|     Scotland|
|   10|     Scotland|
|  150|United States|
|   50|United States|
|   50|United States|
|  150|      England|
|   50|      England|
|   50|      England|
+-----+-------------+
only showing top 20 rows

In [9]:
# Testing the temp view created
spark.sql("select * from breweries_per_type").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+------------+
|count|brewery_type|
+-----+------------+
|    6|        nano|
|    3|    contract|
|    9|      closed|
|    3|  proprietor|
|   27|     taproom|
|  435|       micro|
|   63|       large|
|   21|    regional|
|   90|         bar|
|  273|     brewpub|
+-----+------------+